In [3]:
import autokeras as ak

import numpy as np
from numpy import load
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import random

In [4]:
real_data = load('real.npz')
fake_data = load('fake.npz')

Tx = 1998
Nfreq = 101

# add the real and fake data together with labels 0 for fake and 1 for real
data = np.concatenate((real_data['x'], fake_data['x']))
labels = np.concatenate((np.ones(real_data['x'].shape[0]), np.zeros(fake_data['x'].shape[0])))

# shuffle the data
random.shuffle(data)

# scale the data
scaler = MinMaxScaler()
spectrograms = np.nan_to_num(data)
data = scaler.fit_transform(spectrograms.reshape(-1, Tx * Nfreq)).reshape(-1, Tx, Nfreq, 1)


In [5]:
X_train , X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], Tx, Nfreq, 1)
X_test = X_test.reshape(X_test.shape[0], Tx, Nfreq, 1)

clf = ak.ImageClassifier(overwrite=True, max_trials=2)
clf.fit(X_train, y_train, epochs=10)

# Evaluate the accuracy of the found model
print(clf.evaluate(X_test, y_test))

model = clf.export_model()

model.summary()

try:
    model.save("model_autokeras", save_format="tf")
except Exception:
    model.save("model_autokeras.h5")